In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer

# 1. CARREGAMENTO E LIMPEZA INICIAL
df = pd.read_csv("DENGBR25.csv", low_memory=False)

# Filtramos apenas casos confirmados (10, 11, 12). Removemos o 8 (Inconclusivo) e nulos.
df = df[df['CLASSI_FIN'].isin([10.0, 11.0, 12.0])].copy()

# 2. ENGENHARIA DE ATRIBUTOS (O segredo do R2)
df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'], errors='coerce')
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'], errors='coerce')
df['DIAS_ESPERA'] = (df['DT_NOTIFIC'] - df['DT_SIN_PRI']).dt.days

# Corrigir erros de data (dias negativos ou extremos)
df.loc[(df['DIAS_ESPERA'] < 0) | (df['DIAS_ESPERA'] > 30), 'DIAS_ESPERA'] = df['DIAS_ESPERA'].median()

# 3. REMOÇÃO DE COLUNAS (Lógica Restrita)
drop_patterns = [
    "tp_", "id_", "cod", "codigo", "nu_", "co_", "dt_", "data", "ano", "semana",
    "uf", "municip", "regiao", "pais", "notifica", "notific", "digita", "atualiza",
    "sistema", "fluxo", "origem", "migr", "versao", "status", "lote", "usuario", 
    "login", "sem_", "cs_escol_n", "hospitaliz", "comuninf", "evolucao", 
    "doenca_tra", "nduplic_n", "cs_flxret", "flxrecebi", "tpautocto"
]

def is_to_remove(col_name):
    col = col_name.lower()
    if col in ['classi_fin', 'dias_espera']: return False
    for p in drop_patterns:
        if col == p or col.startswith(p) or col.endswith(p) or p in col.split('_'):
            return True
    return False

cols_to_drop = [c for c in df.columns if is_to_remove(c)]
df_ml = df.drop(columns=cols_to_drop)

# 4. TRATAMENTO DE VALORES (Transformar 1=Sim, 2=Não em 1 e 0)
for col in df_ml.columns:
    if col not in ['CLASSI_FIN', 'DIAS_ESPERA']:
        # Mapeamos 1 para Sim, o resto para 0 (Não/Ignorado)
        df_ml[col] = df_ml[col].apply(lambda x: 1 if x == 1.0 else 0)

# 5. DEFINIÇÃO DO TARGET PARA REGRESSÃO (Escala de Gravidade)
# Mapeamos para uma escala linear para que o R2 faça sentido:
# 10 (Dengue) -> 1 | 11 (Alarme) -> 2 | 12 (Grave) -> 3
map_gravidade = {10.0: 1, 11.0: 2, 12.0: 3}
y = df_ml['CLASSI_FIN'].map(map_gravidade)
X = df_ml.drop(columns=['CLASSI_FIN'])

# 6. TREINAMENTO E R2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Usamos RandomForestRegressor para prever o "Score de Gravidade"
# n_jobs=-1 usa todos os núcleos do seu PC (essencial para 1 milhão de linhas)
model = RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# Avaliação
y_pred = model.predict(X_test)
print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"Erro Médio Absoluto (MAE): {mean_absolute_error(y_test, y_pred):.4f}")

R2 Score: 0.8994
Erro Médio Absoluto (MAE): 0.0052
